In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# GET FEATURE FILES FROM GDRIVE 
# COLAB ONLY - MOVE FEATURE FILES TO WORKING DIRECTORY
# 
import os
import gc
import shutil

if os.getenv("COLAB_RELEASE_TAG") and not os.path.exists(WORKING_DIR):
  !mkdir '/content/asl-work'

if os.getenv("COLAB_RELEASE_TAG"):
   # shutil.copy(f"/content/drive/MyDrive/GaggleSignLang/feature_data{FRAMES_OUT}.npy", f"{WORKING_DIR}")
    shutil.copy(f"/content/drive/MyDrive/GaggleSignLang/feature_labels.npy", f"{WORKING_DIR}")
    shutil.copy(f"/content/drive/MyDrive/GaggleSignLang/sign_to_prediction_index_map.json", f"{WORKING_DIR}")



In [9]:
# Jupiter  MacOS
BASE_DIR = "/Users/johnhanratty/ASLtest/asl-signs"  #"/Users/johnhanratty/ASLtest/asl-signs"
WORKING_DIR = "/Users/johnhanratty/ASLtest"
ARCHIVE_DIR = "/Users/johnhanratty/ASLtest"
MODEL_DIR = "/Users/johnhanratty/ASLtest/models"

# !pip install nb_black --quiet
# %load_ext lab_black

# Colab
# BASE_DIR = "/content/asl-signs"   #"/content/drive/MyDrive/GaggleSignLang/asl-signs"
# WORKING_DIR = "/content/asl-work"
# ARCHIVE_DIR = "/content/drive/MyDrive/GaggleSignLang"
# MODEL_DIR = "/content/drive/MyDrive/GaggleSignLang"
# !pip install nb_black --quiet
# print('-----ok')
# %load_ext nb_black

# KAGGLE
# BASE_DIR = "/kaggle/input/asl-signs"
# WORKING_DIR = "/kaggle/working"
# ARCHIVE_DIR = "/kaggle/working"
# MODEL_DIR  = "/kaggle/working"
# !pip install nb_black --quiet --root-user-action=ignore
# %load_ext lab_black

import os
import gc
import shutil
import time

import json
from tqdm import tqdm
import numpy as np
import pandas as pd
import pickle
from random import seed, sample

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize

import warnings
warnings.filterwarnings(action='ignore')

LANDMARK_FILES_DIR = f'{BASE_DIR}/train_landmark_files'
TRAIN_FILE = f"{BASE_DIR}/train.csv"

FRAMES_OUT = 32 # 16
PTS_IN_FRAME = 345
DIMC = [0,1,2]
DIMS = len(DIMC)
WORKERS = 0   # dataoader work var  0 for MAC, 4 for online

  


print('done')

done


In [16]:
def test(x):
    aa = np.array([1,2,3,4,5,6,7,8,9])
    bb = np.concatenate([aa.reshape(3,3),aa.reshape(3,3)], axis=0)
    return aa, bb

a, b = test(1)
print(a)
print(b)
    

[1 2 3 4 5 6 7 8 9]
[[1 2 3]
 [4 5 6]
 [7 8 9]
 [1 2 3]
 [4 5 6]
 [7 8 9]]


In [4]:
ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [5]:
#FEATUREGEN MODEL
ROWS_PER_FRAME = 543  # combined face, lefth, pose, righth

# FILTER FEATURES IN EACH FRAME  - FACE, POSE & HANDs
class FeatureGen(nn.Module):
    def __init__(self):
        super(FeatureGen, self).__init__()
        pass
    
    def forward(self, x):
        # FILTER TO SPECIFIED FRAMES (FRAMES_OUT)
        seed(24)
        x = np.array(x)
        n_frames = x.shape[0]
        # Trim to # of frames to FRAMES_OUT
        if n_frames > FRAMES_OUT:
            idx = sorted((sample(range(0, n_frames), FRAMES_OUT)))
            x=x[idx,:,:]
        n_frames = x.shape[0]
        # FLATTENING ROWS BY TYPE and CONCATENATING TO ONE ROW PER FRAME 3D (XYZ)
        # INPUT NUMPY, TORCH OUTPUT

        # Grab data type (e.g. one point on hand) by selecting rows for each frame
        # face_x = x[:,:468,:].contiguous().view(-1, 468*3)
        lips_idx = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 95, 88, 178, 87, 14, 317, 402, 318, 324, 146, 91, 181, 84, 17, 314, 405, 321, 375]
        lips_x = x[:, lips_idx,:].reshape(-1, len(lips_idx)*3)
        lefth_x = x[:,468:489,:].reshape(-1, 21*3)
        pose_x = x[:,489:522,:].reshape(-1, 33*3)
        righth_x = x[:,522:,:].reshape(-1, 21*3)

        if np.isnan(lefth_x).sum() < np.isnan(righth_x).sum():
            prime_x = lefth_x
            second_x = righth_x
        else:
            prime_x = righth_x.reshape(righth_x.shape[0], -1, DIMS)
            prime_x[:,:,0] = np.add(np.nanmax(prime_x[:,:,0], axis=1).reshape(-1,1),
                                    prime_x[:, :, 0])
            prime_x = prime_x.reshape(prime_x.shape[0],-1)
            
            second_x = lefth_x.reshape(lefth_x.shape[0], -1, DIMS)
            second_x[:,:,0] = np.subtract(np.nanmax(second_x[:,:,0], axis=1).reshape(-1,1),
                                          second_x[:, :, 0])
            second_x = second_x.reshape(second_x.shape[0],-1)
            
        # ?? remove empty frames ???
        
        # flatten types into one row per frame
        xfeat = np.full([FRAMES_OUT, PTS_IN_FRAME], np.nan)
        offset = (FRAMES_OUT - n_frames) // 2  # center frames in output data in each frame in video
        xfeat[offset:n_frames+offset,:] = np.concatenate([lips_x, prime_x, pose_x, second_x], axis=1)  # concatenate types
        
        
        def distDiff(ds, ref, pts):
            ds = ds.reshape(ds.shape[0],  -1, DIMS)
            d = np.hstack([np.nanmean(ds[:, pts, :], axis=0), 
                           np.nanmedian(ds[:, pts, :], axis=0), 
                           np.nanmax(ds[:, pts, :], axis=0), 
                           np.nanmin(ds[:, pts, :], axis=0),
                           np.nanvar(ds[:, pts, :], axis=0)
                           ]) 
            d = d.reshape(1, -1) 
            # NORMALIZE
           # d = (d - np.nanmean(d, keepdims=True)) / np.nanstd(d, keepdims=True) # -1 to 1
            d = np.nan_to_num(d, copy=False)  # replace NaN after normalization
            return d
        
        d1 = distDiff(xfeat, 40, [44, 48, 52, 56, 60, 43, 46, 50, 54, 58])
        d2 = distDiff(xfeat, 40, [98, 102, 106, 110, 114, 97, 102, 106, 110, 114])
        d3 = distDiff(xfeat, 60, [73, 80, 81, 76, 77, 68, 69, 70, 71, 75, 74])
        d4 = distDiff(xfeat, 5,  [0, 4, 8, 12, 16, 20, 24, 28, 32, 36])
        
        return np.concatenate([d1,d2,d3,d4], axis=1)


In [15]:

## PROCESS EACH ROW (ONE PARQUET PER ROW)
def convert_row(row):
    x = load_relevant_data_subset(os.path.join(BASE_DIR, row[1].path))
    x = feature_converter(torch.tensor(x))
    return x, row[1].label

## LOOP THROUGH PARQUET FILES LISTED IN TRAIN FILE
##  SAVE RESULTS 
def convert_and_save_data():
    label_map = json.load(open(f"{BASE_DIR}/sign_to_prediction_index_map.json", "r"))
    df = pd.read_csv(TRAIN_FILE)
    df['label'] = df['sign'].map(label_map)
    
    print("Convert&Save", df.shape)
    #### FOR TESTING #################
    #df = df[0:20]
    ##################################

    npdata = np.zeros((df.shape[0], 615))  #150+150+165+150

    nplabels = np.zeros(df.shape[0])
    
    results = map(convert_row, df.iterrows())
    for i, (x,y) in tqdm(enumerate(results), total=df.shape[0]):
            npdata[i,:] = x
            nplabels[i] = y
    return npdata, nplabels
 

feature_converter = FeatureGen()
datax, datay = convert_and_save_data()



Convert&Save (94477, 5)


100%|████████████████████████████████████| 94477/94477 [12:40<00:00, 124.21it/s]


In [16]:
# Save dataset
np.save(f"{WORKING_DIR}/cnn_data{FRAMES_OUT}.npy", datax)
np.save(f"{WORKING_DIR}/cnn_labels.npy", datay)
 

In [6]:
# Load dataset
datax = np.load(f"{WORKING_DIR}/cnn_data{FRAMES_OUT}.npy")
datay = np.load(f"{WORKING_DIR}/cnn_labels.npy") 



In [13]:
#MODEL
### NEW SEPARATED INPUTS
class ASLData(Dataset):
    def __init__(self,datax,datay):
        self.datax = datax
        self.datay = datay

    def __getitem__(self, index):
        return self.datax[index, :], self.datay[index]

    def __len__(self):
        return len(self.datay)

# https://towardsdatascience.com/pytorch-tabular-multiclass-classification-9f8211a123ab
class ASLModel(nn.Module):
    def __init__(self, p):
        super(ASLModel, self).__init__()
        
        # DATA in [1, 615] per video
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(p)
        self.relu = nn.ReLU()

        L1OUT = 512  #1024 was ok
        L2OUT = 512

        self.layer_ph = nn.Linear(150, L1OUT)
        self.batchnorm_ph = nn.BatchNorm1d(L1OUT)
        
        self.layer_sh = nn.Linear(150, L1OUT)
        self.batchnorm_sh = nn.BatchNorm1d(L1OUT)
 
        self.layer_po = nn.Linear(165, L1OUT)
        self.batchnorm_po = nn.BatchNorm1d(L1OUT)
 
        self.layer_li = nn.Linear(150, L1OUT)
        self.batchnorm_li = nn.BatchNorm1d(L1OUT) 
 
        self.layer1 = nn.Linear(4*L1OUT, L2OUT)
        self.batchnorm1 = nn.BatchNorm1d(L2OUT)

        self.layerFC = nn.Linear(L2OUT, 250)
        self.softmax = nn.Softmax()
 
        
    def forward(self, x):
        x = torch.tensor(x).float()
        phand = x[:,0:150]            
        shand = x[:,150:300]
        pose =  x[:,300:465]  
        lips =  x[:,465:615]  
        
        ph = self.flatten(torch.tensor(phand).float()) 
        ph = self.layer_ph(ph)
        ph = self.batchnorm_ph(ph)
        ph = self.relu(ph)
        ph = self.dropout(ph)

        sh = self.flatten(torch.tensor(shand).float())       
        sh = self.layer_sh(sh)
        sh = self.batchnorm_sh(sh)
        sh = self.relu(sh)
        sh = self.dropout(sh)
       
        po = self.flatten(torch.tensor(pose).float())       
        po = self.layer_po(po)
        po = self.batchnorm_po(po)
        po = self.relu(po)
        po = self.dropout(po)
        
        li = self.flatten(torch.tensor(lips).float())       
        li = self.layer_li(li)
        li = self.batchnorm_li(li)
        li = self.relu(li)
        li = self.dropout(li)

        x = torch.cat((ph.view(ph.size(0), -1),
                       sh.view(sh.size(0), -1),
                       po.view(po.size(0), -1),
                       li.view(li.size(0), -1)), dim=1)
        # x = self.batchnorm0(x)
        x = self.layer1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.layerFC(x)
       # x = self.softmax(x)

        return torch.tensor(x).float()


In [ ]:
## MULTI TRAINING
# !!! TRAINING DOES NOT RUN ON MAC OS - (cuda)
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("++++using GPU++++")
else:
  device = torch.device("cpu")
  print("++++using CPU++++")

EPOCHS = 10
BATCH_SIZE = 64
start_time = time.perf_counter()

#datax = datax.reshape(datax.shape[0],datax.shape[1], -1) #.swapaxes(1,2)
#datax = torch.tensor(datax)  # Convert to Torch Tensor
datax = torch.tensor(datax)  # Convert to Torch Tensor

trainx, testx, trainy, testy = train_test_split(datax, datay, test_size=0.15, random_state=42)
train_data = ASLData(trainx, trainy)
valid_data = ASLData(testx, testy)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, num_workers=WORKERS, shuffle=True)
val_loader = DataLoader(valid_data, batch_size=BATCH_SIZE, num_workers=WORKERS, shuffle=False)
model = ASLModel(0.2).to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()
sched = torch.optim.lr_scheduler.StepLR(opt, step_size=300, gamma=0.95)
for i in range(EPOCHS):
    model.train()
    
    train_loss_sum = 0.
    train_correct = 0
    train_total = 0
    train_bar = train_loader
    for x,y in train_bar:
        x = torch.Tensor(x).float().to(device)
        y = torch.Tensor(y).long().to(device) 
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        opt.step()
        opt.zero_grad()
        
        train_loss_sum += loss.item()
        train_correct += np.sum((np.argmax(y_pred.detach().cpu().numpy(), axis=1) == y.cpu().numpy()))
        train_total += 1
        sched.step()
        
    val_loss_sum = 0.
    val_correct = 0
    val_total = 0
    model.eval()
    for x,y in val_loader:
        x = torch.Tensor(x).float().to(device)
        y = torch.Tensor(y).long().to(device)
        
        with torch.no_grad():
            y_pred = model(x)
            loss = criterion(y_pred, y)
            val_loss_sum += loss.item()
            val_correct += np.sum((np.argmax(y_pred.cpu().numpy(), axis=1) == y.cpu().numpy()))
            val_total += 1
    print(f"DIM={DIMS} FRAMES={FRAMES_OUT}, FEAT={PTS_IN_FRAME}")                          
    print(f"Epoch:{i} > Train Loss: {(train_loss_sum/train_total):.04f}, Train Acc: {train_correct/len(train_data):0.04f}")
    print(f"Epoch:{i} > Val Loss: {(val_loss_sum/val_total):.04f}, Val Acc: {val_correct/len(valid_data):0.04f}")
    print("="*50)

# Save the pytorch model
PATH = f"{ARCHIVE_DIR}/models/modelccn{FRAMES_OUT}flat.sd"
torch.save(model.state_dict(), PATH)

print("#### ELAPSED TIME:", time.perf_counter()-start_time)



++++using CPU++++
DIM=3 FRAMES=32, FEAT=345
Epoch:0 > Train Loss: 3.5001, Train Acc: 0.2240
Epoch:0 > Val Loss: 2.8052, Val Acc: 0.3326
DIM=3 FRAMES=32, FEAT=345
Epoch:1 > Train Loss: 2.3996, Train Acc: 0.4104
Epoch:1 > Val Loss: 2.4687, Val Acc: 0.4014
DIM=3 FRAMES=32, FEAT=345
Epoch:2 > Train Loss: 2.0425, Train Acc: 0.4889
Epoch:2 > Val Loss: 1.9487, Val Acc: 0.5261
DIM=3 FRAMES=32, FEAT=345
Epoch:3 > Train Loss: 1.8168, Train Acc: 0.5397
Epoch:3 > Val Loss: 1.7693, Val Acc: 0.5641
DIM=3 FRAMES=32, FEAT=345
Epoch:4 > Train Loss: 1.6507, Train Acc: 0.5759
Epoch:4 > Val Loss: 1.6598, Val Acc: 0.5948


In [1]:



# ========Full Data - no add Norm, Centered ========
# DIM=3 FRAMES=32, FEAT=345
# Epoch:38 > Train Loss: 0.9867, Train Acc: 0.7321
# Epoch:38 > Val Loss: 1.2608, Val Acc: 0.6988
# ==================================================

# ========Full Data - no additional Normalizeation/Centering=====================================
# DIM=3 FRAMES=32, FEAT=345
# Epoch:39 > Train Loss: 1.4974, Train Acc: 0.6059
# Epoch:39 > Val Loss: 1.7298, Val Acc: 0.5839
# ==================================================

# =====Full no reverse, centered =============================================
# DIM=3 FRAMES=32, FEAT=345
# Epoch:39 > Train Loss: 1.2104, Train Acc: 0.6745
# Epoch:39 > Val Loss: 1.4204, Val Acc: 0.6515
# ==================================================
# Additional Normalize and Nan Step  0.6100

# Torch Single Model Save, Retrieve and Rum

In [31]:
## SAVE MODEL

PATH = f"{ARCHIVE_DIR}/models/modelccn{FRAMES_OUT}flat.pt"
torch.save(model, PATH)



In [32]:
##LOAD MODEL

PATH = f"{ARCHIVE_DIR}/models/modelccn{FRAMES_OUT}flat.pt"
tf_mod = torch.load(PATH)


In [33]:
# SAVE PYTORCH MODELS
class AModel(nn.Module):
    def __init__(self):
        super(AModel, self).__init__()
        
        self.InputFormat = FeatureGen() #feature_converter
        self.InferModel = tf_mod
        self.InferModel.eval()
    
    def forward(self, x):
        x1,x2,x3,x4 = self.InputFormat(x)
        pred = self.InferModel(x1,x2,x3,x4)
        return pred

mod_cnn = AModel()
mod_cnn_pt_path = f"{ARCHIVE_DIR}/models/modelcnn{FRAMES_OUT}test.pt"
torch.save(mod, PATH)



In [34]:

label_map = json.load(open(f"{BASE_DIR}/sign_to_prediction_index_map.json", "r"))
df = pd.read_csv(TRAIN_FILE)
df['label'] = df['sign'].map(label_map)
df.shape

(94477, 5)

In [35]:
Infmodel = torch.load(mod_cnn_pt_path)

d = df[2022:2023]

x = load_relevant_data_subset(os.path.join(BASE_DIR, d['path'].item()))
pred = Infmodel(x)

print("truth:", d.label, d.sign, "prediction=", np.argmax(pred.detach().numpy()))


truth: 2022    56
Name: label, dtype: int64 2022    dance
Name: sign, dtype: object prediction= 56


# TFLITE CONVERT

In [7]:
!pip install tensorflow --quiet --root-user-action=ignore
!pip install tensorflow_probability --quiet --root-user-action=ignore

!pip install onnx-tf --quiet --root-user-action=ignore
!pip install tflite-runtime  --quiet --root-user-action=ignore
import onnx_tf
import tflite_runtime
import onnx
from onnx_tf.backend import prepare
import tensorflow as tf

In [14]:
## CNN MODEL CONVERSION
# pmodel_path = f"{ARCHIVE_DIR}/models/modelccn16flat.sd"
# modelc = ASLModel(.1)
# modelc.load_state_dict(torch.load(py_model_path))
# modelc.eval()

#### Feature Generation ####
genfeat = FeatureGen()
sample_input = torch.rand((50, 543, 3))
onnx_genfeat_path = f"{MODEL_DIR}/model_genfeat.onnx"

torch.onnx.export(
    genfeat,                    # PyTorch Model
    sample_input,              # Input tensor
    onnx_genfeat_path,        # Output file (eg. 'output_model.onnx')
    opset_version=12,         # Operator support version
    input_names=['input'],     # Input tensor name (arbitary)
    output_names=['output'], # Output tensor name (arbitary)
    dynamic_axes={'input' : {0: 'input'}
    }
)
onnx_genfeat_gen = onnx.load(onnx_genfeat_path)
tf_rep = prepare(onnx_genfeat_gen)

#### CNN MODEL ###
# PATH = f"{ARCHIVE_DIR}/models/modelccn{FRAMES_OUT}flat.pt"
# mod = torch.load(PATH)
# tf_genfeat_path = 'f"{MODEL_DIR}/tf_genfeat'
# tf_rep.export_graph(tf_genfeat_path)


# sample_input = torch.rand((50, 615))
# onnx_mod_cnn_path = f"{MODEL_DIR}/model_cnn.onnx"

# torch.onnx.export(
#     mod,                      # PyTorch Model
#     sample_input,             # Input tensor
#     onnx_mod_cnn_path,        # Output file (eg. 'output_model.onnx')
#     opset_version=12,         # Operator support version
#     input_names=['input'],     # Input tensor name (arbitary)
#     output_names=['output'], # Output tensor name (arbitary)
#     dynamic_axes={'input' : {0: 'input'}
#     }
# )
# onnx_mod_cnn_gen = onnx.load(onnx_mod_cnn_path)
# tf_rep = prepare(onnx_mod_cnn_gen)

# tf_mod_cnn_path = 'f"{MODEL_DIR}/tf_mod_cnn'
# tf_rep.export_graph(tf_mod_cnn_path)



================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



RuntimeError: Only tuples, lists and Variables are supported as JIT inputs/outputs. Dictionaries and strings are also accepted, but their usage is not recommended. Here, received an input of unsupported type: numpy.ndarray

In [2]:
import tensorflow as tf

class ASLInferModel(tf.Module):
    def __init__(self):
        super(ASLInferModel, self).__init__()
        
        self.model = tf.saved_model.load(tf_mod_cnn_path)
        self.model.trainable = False
    
    @tf.function(input_signature=[
      tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')
    ])
    def call(self, input):
        output_tensors = {}
        output_tensors['outputs'] = self.model(**{'input': input})['output'][0,:]
        return output_tensors
    
    
mytfmodel = ASLInferModel()
tf.saved_model.save(mytfmodel, 
                    f'{MODEL_DIR}/tf_infer_model', 
                    signatures={'serving_default': mytfmodel.call})



2023-04-29 20:40:49.329196: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


NameError: name 'tf_mod_cnn_path' is not defined

In [ ]:
# CONVERT MODEL TO TFLITE
converter = tf.lite.TFLiteConverter.from_saved_model(tf_mod_cnn_path) # path to the SavedModel directory
tflite_model = converter.convert()
# Save the model.
TF_MODEL_PATH_LITE = f"{MODEL_DIR}/model_cnn.tflite"
with open(TF_MODEL_PATH, 'wb') as f:
  f.write(tflite_model)

In [57]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=TF_MODEL_PATH_LITE)
interpreter.allocate_tensors()

found_signatures = list(interpreter.get_signature_list().keys())
print("found_signatures")
print(found_signatures[0])

prediction_fn = interpreter.get_signature_runner("serving_default")

label_map = json.load(open(f"{BASE_DIR}/sign_to_prediction_index_map.json", "r"))   ###CHANGE
df = pd.read_csv(TRAIN_FILE)
df['label'] = df['sign'].map(label_map)

vid = df.iloc[26].path
x = load_relevant_data_subset(f"{BASE_DIR}/{vid}")
output = prediction_fn(x)
#interpreter.invoke()

sign = np.argmax(output["outputs"])

print(sign)

found_signatures
serving_default


ValueError: Invalid number of inputs provided for running a SignatureDef, expected 0 vs provided 1